# Stage 2: Full Pipeline with Linear Model

# 1. Preprocessing 

## 1.1 Loading the data

In [ ]:
# Imports
import os
import time
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm 
import shutil
from data.util import *
from data.transformer import * 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import RFE,RFECV #importing RFE class from sklearn library
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import validation_curve
from sklearn.pipeline import Pipeline, FeatureUnion

# load training and testing data into memory
root_path = "data3"
X_train, s_train, y_train_str, p_train, i_train = TrainDataLoader(root_path).load()
X_test = TestDataLoader(root_path).load()
X_train, s_train, y_train_str, p_train, i_train = TrainDataLoader(root_path).load()

# X_train: all features (pixels x 3) of all training samples (4146)
# s_train: superclasses of all samples (4146); note ordered
# y_train: classes of all samples (4146); note ordered
# p_train: pole IDs of all samples (4146)
# i_train: indices of all samples (4146); note duplicates
# X_test: all features (pixels x 3) of all testing samples (4293)

# Make sure that you properly encode the CLASSES such that 
# the order in your submission files is correct!
label_enc = LabelEncoder()
label_enc.fit(CLASSES)
y_train = label_enc.transform(y_train_str) # numerical labels
    
# Randomizer
numpy_seed = 0
python_seed = 0
np.random.seed(numpy_seed)
random.seed(python_seed)

# display some numbers
number_of_train_samples = len(X_train)
number_of_classes = len(np.unique(y_train))
number_of_test_samples = len(X_test)
print('number of training samples:', number_of_train_samples)
print('number of testing samples:', number_of_test_samples)
print('number of labels:', number_of_classes)

## 1.2 Class distribution

In [ ]:
# Create a plot showing the class distribution in the training set
fig_distr, ax_distr = plt.subplots()
ax_distr.set_title("The class distribution of the training set")
ax_distr.set_xlabel("Classes")
ax_distr.set_ylabel("Number of samples")

count = countOcc(y_train)
plt.plot([i for i in range(len(count))], count);
fig_distr.savefig(os.path.join('visualization','class_distribution.png'))

## 1.3 Improve contrast (CLAHE)

In [ ]:
X_train_contr = ContrastTransformer().transform(X_train)
X_test_contr = ContrastTransformer().transform(X_test)

# Show effect of improving contrast
image_index = 99
image_ori = Image.fromarray(X_train[image_index])
image_con = Image.fromarray(X_train_contr[image_index])

# create figure and show images
fig_con, ax_con = plt.subplots(1,2)
ax_con[0].imshow(image_ori)
ax_con[1].imshow(image_con)
ax_con[0].set_axis_off()
ax_con[1].set_axis_off()

## 1.4 Resize all images to a fixed resolution

In [ ]:
X_train_resized = ResizeTransformer().transform(X_train)
X_test_resized = ResizeTransformer().transform(X_test)
print(X_train_resized.shape)
print(X_test_resized.shape)

# Show effect of resizing
image_index = 17
image_ori = Image.fromarray(X_train[image_index])
image_res = Image.fromarray(X_train_resized[image_index])
# create figure and show images
fig_res, ax_res = plt.subplots(1,2)
ax_res[0].imshow(image_ori)
ax_res[1].imshow(image_res)
ax_res[0].set_axis_off()
ax_res[1].set_axis_off()

# 2. Feature Extraction

## 2.1. Aspect Ratio

In [ ]:
aspect_ratio = [get_ratio(img) for img in X_train]

plt.ylim(0,5)
plt.xlabel("Class index")
plt.ylabel("Aspect Ratio")
plt.title("Distribution of aspect ratio among training samples.")
plt.scatter(y_train, aspect_ratio, alpha=0.42, s=1)

## 2.1. Color Histograms

In [ ]:
nbinh=3
X_train_hist = colorhistTransformer(nbin=nbinh).transform(X_train)
X_test_hist = colorhistTransformer(nbin=nbinh).transform(X_test)
print(X_train_hist.shape)
print(X_test_hist.shape)

# Show corresponding image
image_index = 99
image_ori = Image.fromarray(X_train[image_index])
# create figure and show images
fig_rgb, ax_rgb = plt.subplots()
ax_rgb.imshow(image_ori)
ax_rgb.set_axis_off()

def image_plot(hist,image,colors):
    plt.figure(figsize=(20,10))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.subplot(1, 2, 2)
    plt.bar(np.arange(len(hist)),hist,width=1,color=colors,linewidth=0)
colors=[]
for r in np.arange(0.5,nbinh+0.5):
    for g in np.arange(0.5,nbinh+0.5):
        for b in np.arange(0.5,nbinh+0.5):
            colors.append((r/nbinh,g/nbinh,b/nbinh))
image_plot(X_train_hist[image_index],X_train[image_index],colors*256)

## 2.2 HOG features

In [ ]:
X_train_hog = HogTransformer().transform(X_train_resized)
X_test_hog = HogTransformer().transform(X_test_resized)
print(X_train_hog.shape)
print(X_test_hog.shape)

# 3. Feature Visualization

## 3.1 Histogram for 1 HOG feature
A histogram that plots the number of samples with certain HOG values for 1 specific HOG feature.

In [ ]:
# Visualize the histogram for a specific feature
feature_index = 9
feature = X_train_hog[:,feature_index]
fig_feat, ax_feat = plt.subplots()
ax_feat.set_title("Histogram for feature {}".format(feature_index))
ax_feat.set_xlabel("HOG values")
ax_feat.set_ylabel("number of samples")
plot_histogram(ax_feat, feature, bins=50)

## 3.2 HOG features for 1 sample

In [ ]:
# visualization of HOG features (NOTE: uses skimage HOG)
from skimage.feature import hog

image_index = 7
image_original = X_train_resized[image_index]
fd, image_hog = hog(image_original, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(8, 8), visualize=True, multichannel=True)

fig_hog, ax_hog = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
ax_hog[0].imshow(image_original)
ax_hog[1].imshow(image_hog)
ax_hog[0].set_axis_off()
ax_hog[1].set_axis_off()
print(fd.shape)

## 3.3 Color histogram for 1 sample
A histogram that plots the number of features with certain HOG values for 1 specific sample. There's also a function implemented to plot histograms for multiple samples. This is to visually check for correlations between samples and try to understand what the poleIDs and IDs play as roles in the dataset.
Note that samples with different poleIDs but identical IDs have (visually) identical histograms. This implies a correlation between these samples.

In [ ]:
# Visualize the histogram for a specific sample
sample_index = 99
sample = X_train_hog[sample_index,:]
image = Image.fromarray(X_train[sample_index])
#image.show() # uncomment to show corresponding images
fig_samp, ax_samp = plt.subplots()
ax_samp.set_title("Histogram for sample {}".format(sample_index))
ax_samp.set_xlabel("HOG values")
ax_samp.set_ylabel("number of features")
plot_histogram(ax_samp, sample, bins=50)
fig_im, ax_im = plt.subplots()
ax_im.imshow(image)
ax_im.set_axis_off()

## 3.4. Feature Combination
Combine HOG and color features.

In [ ]:
clf_lr = LogisticRegression(max_iter=1000, n_jobs=-2)  
rfe = RFE(estimator= clf_lr , step =50)  
# inx=np.random.randint(0,len(X_train)-1,1000)
# fit = rfe.fit(X_train_hog[inx,:], y_train[inx])

X_train_hog_hist = np.concatenate((X_train_hog, X_train_hist), axis=1)
X_test_hog_hist = np.concatenate((X_test_hog, X_test_hist), axis=1)
print(X_train_hog_hist.shape)
print(X_test_hog_hist.shape)

## 3.4. Feature Correlations
We can also look at the correlations between feature pairs. Multiple correlation coefficient calculation methods are possible.

In [ ]:
# We can also look at the correlations between feature pairs 
# and between the features and the labels

# turn numpy X_train into pd dataframe and add column with labels
row_names = [i for i in range(1,number_of_train_samples+1)]
column_names = ['HOG'+ str(i) for i in range(1,X_train_hog.shape[1]+1)]
column_names += ['COL' + str(i) for i in range(1,nbinh**3+1) ]
pd_X_train_hog_hist = pd.DataFrame(data=X_train_hog_hist, 
                                  index=row_names,
                                        columns=column_names)
pd_X_train_hog_hist['class'] = y_train
pd_X_train_hog_hist.head()
plot_correlation_matrix(pd_X_train_hog_hist)

In [ ]:
# You can check the # of ftrs eleminated by transformming 
for t in [.75]:#,.8,.85,.9
    X_train_corrTrunc = CorrMatrixTransformer(treshold=t).transform(X_train_hog, y_train)
    print(f'Threshold value {t}')
    print(X_train_corrTrunc.shape)

# 4. Pipeline

In [4]:
# cross validation strategy -- stratified and grouped
cv_indices = []
folds = 5
sgkf = stratified_group_k_fold(X_train, y_train, p_train, k=folds)
for train_fold, test_fold in sgkf:
    cv_indices.append((train_fold, test_fold))

In [ ]:
# tuning ranges
boolean = [True, False]
lr_weights = ['balanced', None]
lr_C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
lr_class = ['ovr', 'multinomial']
hog_sizes = [64, 32]
hog_bins = [5, 9, 13]
hist_bin= [2, 3, 4]
lda_solvers = ['svd', 'lsqr', 'eigen']
lda_shrinkage = np.arange(0, 1, 0.2)
corr_treshold = [.8,.85,.9]
corr_treshold = [.02]
corr_method_list = ['pearson', 'kendall', 'spearman'] 

In [ ]:
# different pipeline sections
pipeline_hog = Pipeline([('resize', ResizeTransformer()),
                        ('hog', HogTransformer(winSize=32, blockSize=32, 
                                    blockStride=2, cellSize=8,
                                    nbins=15))
                        ])
pipeline_colorhist = Pipeline([('colorhist', colorhistTransformer(nbin=4))])
pipeline_aspect_ratio = Pipeline([('aspect_ratio', AspectRatioTransformer())])

# combinated
union = FeatureUnion([('HOG', pipeline_hog), 
                     ('COLOR', pipeline_colorhist),
                     ('REST', pipeline_aspect_ratio)
                     ])

pipeline = Pipeline([('union', union),
                    ('scalar', StandardScaler()),
                    ('classify_lr', LogisticRegression(max_iter=1000, n_jobs=-2))
                    ])


# parameter grid
param_grid = {'classify_lr__C':[.1]} # short search

# perform grid search
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=cv_indices, 
                            scoring=neg_logloss_scorer, verbose=True, 
                            n_jobs=-1, return_train_score=True)
grid_search.fit(X_train, y_train, groups=p_train)
show_grid_scores(grid_search)

# get best model and calculate score
estimator = get_best_model(grid_search)
scores = cross_validate(estimator, X_train, y_train, groups=p_train, 
                        cv=cv_indices, scoring=neg_logloss_scorer, return_train_score=True)
show_scores(scores)

show_misclassifications(pipeline, X_train, y_train, cv_indices, 1, 20)

# 5. Model Evaluation

## 5.1. Validation Curve

In [ ]:
# validation curves for 2 dimensional grid search, containing C
plot_grid_search(grid_search.cv_results_, param_grid)

In [ ]:
# validation curve for 1 dimensional grid search, any hyperparameter
# Note: this performs a gridsearch and is very slow
param_name = 'classify_lr__C'
param_range = param_grid[param_name]
valid_curv = validation_curve(estimator, X_train, y_train, param_name=param_name, param_range=param_range, 
                                cv=cv_indices, groups=p_train, scoring=neg_logloss_scorer, 
                                    n_jobs=-2)
plot_validation_curve(valid_curv, param_name, param_range, xscale="log")

## 5.2. Learning Curve

In [ ]:
# calculate and plot learning curve
learn_curv = learning_curve(estimator, X_train, y_train, train_sizes=np.linspace(.1, 1.0, 5),
                                cv=cv_indices, groups=p_train, scoring=neg_logloss_scorer, 
                                    n_jobs=-2, shuffle=True)
plot_learning_curve(learn_curv)

## 5.3. Misclassifications

In [ ]:
# plot confusion matrix and corresponding misclassified images for a certain fold
show_misclassifications(estimator, X_train, y_train, cv_indices, fold_number=1, N=20)

# 6. Submission

In [ ]:
# train best model using all training data
estimator.fit(X_train, y_train)
# Here is where we create the submission for your estimator
output_probabilities = estimator.predict_proba(X_test)
create_submission(output_probabilities, 'submission.csv')